In [44]:
import csv 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SpotifySongs").getOrCreate()

df = spark.read.csv("../assets/data/spotify-2023.csv", header=True, inferSchema=True)

In [36]:
df.dtypes

[('track_name', 'string'),
 ('artist(s)_name', 'string'),
 ('artist_count', 'int'),
 ('released_year', 'int'),
 ('released_month', 'int'),
 ('released_day', 'int'),
 ('in_spotify_playlists', 'int'),
 ('in_spotify_charts', 'int'),
 ('streams', 'string'),
 ('in_apple_playlists', 'int'),
 ('in_apple_charts', 'int'),
 ('in_deezer_playlists', 'string'),
 ('in_deezer_charts', 'int'),
 ('in_shazam_charts', 'string'),
 ('bpm', 'int'),
 ('key', 'string'),
 ('mode', 'string'),
 ('danceability_%', 'int'),
 ('valence_%', 'int'),
 ('energy_%', 'int'),
 ('acousticness_%', 'int'),
 ('instrumentalness_%', 'int'),
 ('liveness_%', 'int'),
 ('speechiness_%', 'int')]

1. ## Most Streamed & Highest Ranking Tracks:

- Identify the top 5 tracks with the highest number of streams in 2023.

In [24]:
top_5_by_streams = df.select("track_name", "artist(s)_name", "streams").orderBy(df.streams.cast('int').desc()).limit(5)

top_5_by_streams.show()

top_5_by_streams_released_2023 = df.select("track_name", "artist(s)_name", "streams").filter(df.released_year == 2023).orderBy(df.streams.cast('int').desc()).limit(5)

top_5_by_streams_released_2023.show()


+-----------------+--------------+----------+
|       track_name|artist(s)_name|   streams|
+-----------------+--------------+----------+
|Take Me To Church|        Hozier|2135158446|
|          Circles|   Post Malone|2132335812|
|    Love Yourself| Justin Bieber|2123309722|
|        All of Me|   John Legend|2086124197|
|   Counting Stars|   OneRepublic|2011464183|
+-----------------+--------------+----------+

+--------------------+--------------------+----------+
|          track_name|      artist(s)_name|   streams|
+--------------------+--------------------+----------+
|             Flowers|         Miley Cyrus|1316855716|
|     Ella Baila Sola|Eslabon Armado, P...| 725980112|
|Shakira: Bzrp Mus...|   Shakira, Bizarrap| 721975598|
|                 TQG|    Karol G, Shakira| 618990393|
|     La Bebe - Remix|Peso Pluma, Yng L...| 553634067|
+--------------------+--------------------+----------+



- Identify the top 5 ranking tracks across all platforms.

In [25]:
chart_presence_df = df.withColumn("total_charts_presence", df.in_spotify_charts + df.in_apple_charts + df.in_deezer_charts + df.in_shazam_charts).select("track_name", "artist(s)_name", "total_charts_presence")

chart_presence_df.orderBy(chart_presence_df.total_charts_presence.desc()).limit(5).show()

+--------------------+-----------------+---------------------+
|          track_name|   artist(s)_name|total_charts_presence|
+--------------------+-----------------+---------------------+
|             vampire|   Olivia Rodrigo|               1283.0|
|            Sprinter|Dave, Central Cee|               1267.0|
|            fukumean|            Gunna|               1257.0|
|Seven (feat. Latt...| Latto, Jung Kook|               1246.0|
|              Tattoo|           Loreen|               1104.0|
+--------------------+-----------------+---------------------+



- Is there any crossover between these two findings?

There does not seem to be any crossover between the top 5 most streamed songs and the songs that appeared in the most charts. 

2. ## Artists with Most Tracks in Spotify Charts:

Group by the artist's name and count how many of their tracks made it to the Spotify charts. Display the top 10 artists.

In [35]:
most_tracks_in_spotify_charts = df.select('artist(s)_name').filter(df.in_spotify_charts != 0).groupBy("artist(s)_name").count().orderBy("count", ascending=False).limit(10)
most_tracks_in_spotify_charts.show()

+--------------+-----+
|artist(s)_name|count|
+--------------+-----+
|  Taylor Swift|   22|
|     Bad Bunny|   12|
|  Harry Styles|   10|
|    The Weeknd|    8|
|          Feid|    7|
| Morgan Wallen|    6|
|    Ed Sheeran|    6|
|      NewJeans|    6|
|Olivia Rodrigo|    5|
|       Karol G|    4|
+--------------+-----+



3. ## Filter and Select Attributes of Danceable Tracks:
- Extract tracks with a danceability percentage of more than 80% and display their name, artist, bpm (beats per minute), and danceability percentage.
- Identify any possible correlation between danceability and bpm.

In [45]:
danceable_tracks = df.select("track_name", "artist(s)_name", "bpm", "danceability_%").filter(df["danceability_%"] > 80).orderBy(df["danceability_%"].desc())
danceable_tracks.show()
correlation = danceable_tracks.stat.corr('bpm', 'danceability_%')

print("Correlation between 'bpm' and 'danceability_%':", correlation)

+--------------------+--------------------+---+--------------+
|          track_name|      artist(s)_name|bpm|danceability_%|
+--------------------+--------------------+---+--------------+
|                Peru|Ed Sheeran, Fireb...|108|            96|
| The Real Slim Shady|              Eminem|104|            95|
|             Players|           Coi Leray|105|            95|
|               CAIRO|Karol G, Ovy On T...|115|            95|
|   Super Freaky Girl|         Nicki Minaj|133|            95|
|           Starlight|                Dave|124|            95|
|            Ai Preto|L7nnon, DJ Biel d...|130|            95|
|         Slut Me Out|          NLE Choppa|121|            94|
|        Shorty Party|Cartel De Santa, ...| 96|            93|
|Gol Bolinha, Gol ...| Mc Pedrinho, DJ 900|133|            93|
|    Forgot About Dre|     Eminem, Dr. Dre|134|            93|
|sentaDONA (Remix) s2|Lu��sa Sonza, MC ...|135|            93|
|  Privileged Rappers|    Drake, 21 Savage|144|        